In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import pandas as pd
from colorama import Fore
import pickle as pkl
import data, viz
from style import *
out_dir = 'results/jun17_1' # jun2_9

# compare all models

In [2]:
def load_results(out_dir):
    r = []
    for fname in os.listdir(out_dir):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        metrics = {k: d['cv'][k] for k in d['cv'].keys() if not 'curve' in k}
        out = {k: np.mean(metrics[k]) for k in metrics}
        out.update({k + '_std': np.std(metrics[k]) for k in metrics})
        for test in ['test1', 'test2']:
            metrics_test = {k: d[test][k] for k in d[test].keys() if not 'curve' in k}
            out.update({k + f'_{test}': np.mean(metrics_test[k]) for k in metrics_test})
            out.update({k + f'_{test}_std': np.std(metrics_test[k]) for k in metrics_test})
        
        
        out['model_type'] = fname.replace('.pkl', '') #d['model_type']
        imp_mat = np.array(d['imps']['imps'])
        imp_mu = imp_mat.mean(axis=0)
        imp_sd = imp_mat.std(axis=0)
        
        
        feat_names = d['feat_names_selected']
        out.update({feat_names[i] + '_f': imp_mu[i] for i in range(len(feat_names))})
        out.update({feat_names[i]+'_std_f': imp_sd[i] for i in range(len(feat_names))})
        r.append(pd.Series(out))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('model_type')
    return r

results = load_results(out_dir)
# print(results.keys())

## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [3]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)

# filter out if it always predits positive
r = r[r['tn'] > 0]

r = r.sort_values(by=['fn', 'tn'], ascending=True)
# r = r[['fn', 'tn', 'fp', 'tp', 'specificity', 'sensitivity', 'roc_auc', 'precision',
#        'f1', 'balanced_accuracy', 'accuracy']]
r = r[['fn', 'tn', 'fp', 'tp',
       'specificity', 'sensitivity', 'roc_auc', 'precision',
       'f1', 'balanced_accuracy', 'accuracy',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

,fn,tn,fp,tp,specificity,sensitivity,roc_auc,precision,f1,balanced_accuracy,accuracy,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
model_type,,,,,,,,,,,,,,,,,,,
logistic_select_stab_lasso=5_ros=100,0.25,15.5,1960.25,32.25,0.18,0.99,0.883,0.016,0.032,0.499,0.024,0.25,39.25,3898.75,72.75,0,0.5,2125.5,62
logistic_select_stab_lasso=6_ros=100,0.25,19.5,1956.25,32.25,0.229,0.99,0.886,0.016,0.032,0.5,0.026,0.25,48.5,3889.5,72.75,0,2.5,2123.5,62
logistic_select_stab_lasso=7_ros=100,0.25,42.75,1933,32.25,0.369,0.99,0.873,0.016,0.032,0.506,0.037,0.25,102.75,3835.25,72.75,0,21.75,2104.25,62
logistic_select_stab_lasso=5_sample_weights=100,5,1485,490.75,27.5,0.982,0.844,0.886,0.053,0.099,0.798,0.753,15,2947,991,58,7,1417.5,708.5,55
logistic_select_stab_lasso=7_sample_weights=100,5.25,1540,435.75,27.25,0.983,0.838,0.884,0.059,0.11,0.809,0.781,16,3064.5,873.5,57,8.25,1518.25,607.75,53.75
logistic_select_stab_lasso=6_sample_weights=100,5.25,1544.25,431.5,27.25,0.983,0.839,0.889,0.06,0.111,0.81,0.783,16.75,3082.75,855.25,56.25,9.5,1515.75,610.25,52.5
logistic_select_stab_lasso=10_sample_weights=100,5.25,1590.25,385.5,27.25,0.983,0.839,0.886,0.066,0.122,0.822,0.806,18,3150.75,787.25,55,7,1562.75,563.25,55
logistic_select_stab_lasso=26_sample_weights=100,5.25,1595.25,380.5,27.25,0.983,0.84,0.88,0.067,0.124,0.824,0.808,17,3164.75,773.25,56,30,1967,159,32
logistic_select_stab_lasso=10_sample_weights=5,19,1946,29.75,13.5,0.993,0.422,0.897,0.323,0.362,0.703,0.976,57.25,3887,51,15.75,35.5,2063.5,62.5,26.5


**look at feat importances**

In [4]:
r = results
best_model = 'logistic_select_stab_lasso=5_sample_weights=100' #logistic_select_rf=5_smote=5'
# print(r.keys())
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r.sort_values('balanced_accuracy', ascending=False)
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('35')]
# r = r[r.index.str.contains('11')]
# r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('20')]
r = r[r.index.str.contains(best_model)]

r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
# r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank
idxs = r.index
# r.insert(0, 'balanced_acc', results.loc[idxs]['balanced_accuracy'])
# r = r.sort_values('balanced_acc', ascending=False)
# subset = list(r.keys())
# subset.remove('balanced_acc')
# r.to_html('table.html')
# r = r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1) #, subset=subset) # rows differently colored


r = r.transpose()
r = r.reindex(r[best_model].abs().sort_values(ascending=False).index) # sort by abs value
# r = r.sort_values(by='logistic_20_ros=1')
r = r.fillna(0).style.background_gradient(cmap=cm) #, axis=0) #, subset=subset) # rows differently colored
# r = background_gradient(r)
r

model_type,logistic_select_stab_lasso=5_sample_weights=100
GCSScore_Full_yes,-0.821
AbdTrauma_or_SeatBeltSign_yes,0.647
AbdTenderDegree_Mild,-0.564
AbdTenderDegree_Severe,0.338
AbdDistention_yes,0.295
MOI_Motor vehicle collision,0.291
InitSysBPRange,-0.209
AbdDistention_unknown,-0.053
MOI_Motorcycle/ATV/Scooter collision,0.036
VomitWretch_yes,0


# analyze on final test data

In [ ]:
df = data.get_data()
outcome_def = 'iai_intervention'
idxs_test = df.cv_fold.isin([6])
X_test, Y_test = df[idxs_test], df[outcome_def][idxs_test]

In [ ]:
model_name = 'logistic_20_ros=5'
# results = pkl.load(open('results/classify_outcome=y_consec_sig/rf_ros_11.pkl', 'rb'))
results = pkl.load(open(f'results/outcome=iai_intervention/{model_name}.pkl', 'rb'))
preds, preds_proba = viz.visualize_individual_results(results, X_test, Y_test, print_results=False)

# misc analysis

In [ ]:
def count_common_feats(m_cv, model_type='tree'):
    if model_type == 'tree':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            feats_used += list(ks[m_fit.feature_importances_ != 0])
    elif model_type == 'logistic':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            num_feats = 5
            # get top num_feats features with biggest bsolute weights
            idxs = np.abs(m_fit.coef_).flatten().argsort()[-num_feats:][::-1]
            feats_used += list(ks[idxs])
    return sorted(dict(Counter(feats_used)).items(), key=lambda kv: kv[1], reverse=True)
        
count_common_feats(m_cv, model_type)

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=ks)
plt.show()

In [18]:
from sklearn.datasets import load_iris
from skrules import SkopeRules

dataset = load_iris()
feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
clf = SkopeRules(max_depth_duplication=2,
                 n_estimators=30,
                 precision_min=0.3,
                 recall_min=0.1,
                 feature_names=feature_names)

for idx, species in enumerate(dataset.target_names):
    X, y = dataset.data, dataset.target
    clf.fit(X, y == idx)
    rules = clf.rules_[0:3]
    print("Rules for iris", species)
    for rule in rules:
        print(rule)
    print()
    print(20*'=')
    print()

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


Rules for iris setosa
('petal_width <= 0.75', (1.0, 1.0, 4))
('petal_length <= 2.449999988079071', (1.0, 1.0, 26))


Rules for iris versicolor
('petal_length <= 4.950000047683716 and petal_width <= 1.699999988079071 and petal_width > 0.800000011920929', (1.0, 0.9517441860465117, 2))
('petal_length <= 4.8500001430511475 and petal_width > 0.800000011920929', (0.9512195121951219, 0.9512195121951219, 1))
('petal_length <= 4.75 and petal_length > 2.449999988079071', (0.986842105263158, 0.8823366555924695, 2))


Rules for iris virginica
('petal_width > 1.6500000357627869', (0.975609756097561, 0.9302325581395349, 1))
('petal_length > 4.8500001430511475', (0.9512195121951219, 0.9512195121951219, 1))
('petal_length > 4.75 and petal_width > 1.75', (0.9738163517768781, 0.903094866704481, 4))




In [17]:
pip install skope-rules

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
